## Text-gen RNN

### Import relevant libraries

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

### Load text into a one huge string (millions of chars)

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()

In [3]:
print(text[:100])
print(len(text))

Chapter 1


Happy families are all alike; every unhappy family is unhappy in its own
way.

Everythin
1966145


### Create vocabulary : a set of all chars of which 'text' consists

In [4]:
vocab = set(text)

In [5]:
print(vocab)


{'E', 'n', 'O', '5', 'g', "'", '-', 'B', 'q', 'K', 'U', 'Q', 'X', ')', '\n', 'k', 'l', '!', '`', 'e', 'J', 'H', 'a', 'h', 'b', 'p', 'F', 'A', 'N', 'R', 'y', '1', 's', 'L', 'W', 'I', '7', '3', 'M', 'i', 'C', 'P', '6', 'S', '2', 'w', '9', ' ', 'o', ',', '"', 'x', 'm', 'Y', '?', 'v', 'r', '_', '(', ':', 'G', 'D', 'f', '8', 'T', 't', 'j', '0', 'u', '.', 'c', 'Z', 'V', 'd', 'z', ';', '4'}


### Create vocab_to_int and int_to_vocab. These are dictionaries. You won't need it. Hopefully

In [6]:
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))

In [7]:
print(vocab_to_int)
print()
print(int_to_vocab)

{'E': 0, 'b': 24, 'n': 1, '5': 3, 'g': 4, 'U': 10, 'B': 7, 'K': 9, 'q': 8, 'O': 2, 'k': 15, 'l': 16, '!': 17, '`': 18, '4': 76, 'e': 19, 's': 32, 'J': 20, 'a': 22, 'h': 23, 'p': 25, 'R': 29, "'": 5, 'N': 28, 'y': 30, '1': 31, 'W': 34, 'L': 33, 'X': 12, '7': 36, 'I': 35, '9': 46, '.': 69, '3': 37, 'H': 21, 'M': 38, 'i': 39, 'C': 40, 'c': 70, 'P': 41, '6': 42, 'S': 43, 'G': 60, 'w': 45, 'A': 27, '-': 6, ' ': 47, '0': 67, 'o': 48, '"': 50, '2': 44, 'x': 51, 'm': 52, 'Y': 53, 'F': 26, 'r': 56, '(': 58, ':': 59, 'D': 61, '\n': 14, 'f': 62, '?': 54, 'T': 64, 't': 65, '8': 63, 'Z': 71, ',': 49, ';': 75, 'v': 55, 'u': 68, 'j': 66, 'Q': 11, 'V': 72, 'd': 73, 'z': 74, ')': 13, '_': 57}

{0: 'E', 1: 'n', 2: 'O', 3: '5', 4: 'g', 5: "'", 6: '-', 7: 'B', 8: 'q', 9: 'K', 10: 'U', 11: 'Q', 12: 'X', 13: ')', 14: '\n', 15: 'k', 16: 'l', 17: '!', 18: '`', 19: 'e', 20: 'J', 21: 'H', 22: 'a', 23: 'h', 24: 'b', 25: 'p', 26: 'F', 27: 'A', 28: 'N', 29: 'R', 30: 'y', 31: '1', 32: 's', 33: 'L', 34: 'W', 35: 'I'

### Create an iteger representation of 'text' (millions of chars as ints)

unichr(x) - char from unicode int
ord(x) - byte or in from chr() or unichar() respectively

In [8]:
#chars = np.array([ord(c) for c in text], dtype=np.int32)
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)
print(chars[:100])

[40 23 22 25 65 19 56 47 31 14 14 14 21 22 25 25 30 47 62 22 52 39 16 39 19
 32 47 22 56 19 47 22 16 16 47 22 16 39 15 19 75 47 19 55 19 56 30 47 68  1
 23 22 25 25 30 47 62 22 52 39 16 30 47 39 32 47 68  1 23 22 25 25 30 47 39
  1 47 39 65 32 47 48 45  1 14 45 22 30 69 14 14  0 55 19 56 30 65 23 39  1]


## Data split

In [9]:
def split_data(chars_vector,
               samples_per_batch,
               sample_length,
               split_frac=0.9):

    x = chars_vector[:-1]
    y = chars_vector[1:]

    sample_count = len(x)-sample_length+1
    
    x_batches = []
    y_batches = []
    
    start_range = range(0, sample_count, sample_length)

    x_samples = np.array([x[start:start+sample_length] for start in start_range])
    y_samples = np.array([y[start:start+sample_length] for start in start_range])

    if sample_count > samples_per_batch:

        batch_count = len(x_samples) // samples_per_batch
        new_length = batch_count * samples_per_batch
        end_crop_count = len(x_samples)-new_length

        if end_crop_count != 0:
            x_samples = x_samples[:-end_crop_count]
            y_samples = y_samples[:-end_crop_count]

        x_batches = np.array(np.split(x_samples, batch_count))
        y_batches = np.array(np.split(y_samples, batch_count))
        
    else:
        
        x_batches = x_samples
        y_batches = y_samples
        
    
    split_idx = int(len(x_batches)*split_frac)

    train_x, train_y = x_batches[:split_idx], y_batches[:split_idx]
    val_x, val_y = x_batches[split_idx:], y_batches[split_idx:]
    
    return train_x, train_y, val_x, val_y

In [10]:
vector = np.arange(42)
samples_in_batch = 5
sample_length = 3
split_frac = 1.

tx, ty, _, _ = split_data(vector, samples_in_batch, sample_length, split_frac)

print('\ntx:')
print(tx)
print('\nty:')
print(ty)

print(tx.shape)


tx:
[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]
  [ 9 10 11]
  [12 13 14]]

 [[15 16 17]
  [18 19 20]
  [21 22 23]
  [24 25 26]
  [27 28 29]]]

ty:
[[[ 1  2  3]
  [ 4  5  6]
  [ 7  8  9]
  [10 11 12]
  [13 14 15]]

 [[16 17 18]
  [19 20 21]
  [22 23 24]
  [25 26 27]
  [28 29 30]]]
(2, 5, 3)


In [11]:
def get_batch(tx, ty):
    for x, y in zip(tx, ty):
        yield x, y

#### Creating training and validation sets using function defined above

In [12]:
train_x, train_y, val_x, val_y = split_data(chars_vector=chars, 
                                            samples_per_batch=100, 
                                            sample_length=100)

In [13]:
print('train_x.shape: {}'.format(train_x.shape))
print('train_y.shape: {}'.format(train_y.shape))



train_x.shape: (176, 100, 100)
train_y.shape: (176, 100, 100)


In [14]:
train_x[0,:2,:]

array([[40, 23, 22, 25, 65, 19, 56, 47, 31, 14, 14, 14, 21, 22, 25, 25, 30,
        47, 62, 22, 52, 39, 16, 39, 19, 32, 47, 22, 56, 19, 47, 22, 16, 16,
        47, 22, 16, 39, 15, 19, 75, 47, 19, 55, 19, 56, 30, 47, 68,  1, 23,
        22, 25, 25, 30, 47, 62, 22, 52, 39, 16, 30, 47, 39, 32, 47, 68,  1,
        23, 22, 25, 25, 30, 47, 39,  1, 47, 39, 65, 32, 47, 48, 45,  1, 14,
        45, 22, 30, 69, 14, 14,  0, 55, 19, 56, 30, 65, 23, 39,  1],
       [ 4, 47, 45, 22, 32, 47, 39,  1, 47, 70, 48,  1, 62, 68, 32, 39, 48,
         1, 47, 39,  1, 47, 65, 23, 19, 47,  2, 24, 16, 48,  1, 32, 15, 30,
        32,  5, 47, 23, 48, 68, 32, 19, 69, 47, 64, 23, 19, 47, 45, 39, 62,
        19, 47, 23, 22, 73, 14, 73, 39, 32, 70, 48, 55, 19, 56, 19, 73, 47,
        65, 23, 22, 65, 47, 65, 23, 19, 47, 23, 68, 32, 24, 22,  1, 73, 47,
        45, 22, 32, 47, 70, 22, 56, 56, 30, 39,  1,  4, 47, 48,  1]], dtype=int32)

In [15]:
train_y[0, :2, :]

array([[23, 22, 25, 65, 19, 56, 47, 31, 14, 14, 14, 21, 22, 25, 25, 30, 47,
        62, 22, 52, 39, 16, 39, 19, 32, 47, 22, 56, 19, 47, 22, 16, 16, 47,
        22, 16, 39, 15, 19, 75, 47, 19, 55, 19, 56, 30, 47, 68,  1, 23, 22,
        25, 25, 30, 47, 62, 22, 52, 39, 16, 30, 47, 39, 32, 47, 68,  1, 23,
        22, 25, 25, 30, 47, 39,  1, 47, 39, 65, 32, 47, 48, 45,  1, 14, 45,
        22, 30, 69, 14, 14,  0, 55, 19, 56, 30, 65, 23, 39,  1,  4],
       [47, 45, 22, 32, 47, 39,  1, 47, 70, 48,  1, 62, 68, 32, 39, 48,  1,
        47, 39,  1, 47, 65, 23, 19, 47,  2, 24, 16, 48,  1, 32, 15, 30, 32,
         5, 47, 23, 48, 68, 32, 19, 69, 47, 64, 23, 19, 47, 45, 39, 62, 19,
        47, 23, 22, 73, 14, 73, 39, 32, 70, 48, 55, 19, 56, 19, 73, 47, 65,
        23, 22, 65, 47, 65, 23, 19, 47, 23, 68, 32, 24, 22,  1, 73, 47, 45,
        22, 32, 47, 70, 22, 56, 56, 30, 39,  1,  4, 47, 48,  1, 47]], dtype=int32)

### Building the model

In [16]:
def build_rnn(num_classes,
              samples_per_batch=50,
              sample_length=50,
              lstm_size=128, 
              num_layers=2,
              learning_rate=0.001, 
              grad_clip=5, 
              sampling=False):
    
    # When we're using this network for sampling later, we'll be passing in
    # one character at a time, so providing an option for that
    if sampling == True:
        samples_per_batch, sample_length = 1, 1

    tf.reset_default_graph()
    
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [samples_per_batch, sample_length], name='inputs')
    targets = tf.placeholder(tf.int32, [samples_per_batch, sample_length], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # One-hot encoding the input and target characters
    x_one_hot = tf.one_hot(inputs, num_classes)
    y_one_hot = tf.one_hot(targets, num_classes)

    ### Build the RNN layers
    # Use a basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    initial_state = cell.zero_state(samples_per_batch, tf.float32)

    ### Run the data through the RNN layers
    # This makes a list where each element is on step in the sequence
    rnn_inputs = [tf.squeeze(i, squeeze_dims=[1]) for i in tf.split(x_one_hot, sample_length, 1)]
    
    # Run each sequence step through the RNN and collect the outputs
    outputs, state = tf.contrib.rnn.static_rnn(cell, rnn_inputs, initial_state=initial_state)
    final_state = state
    
    # Reshape output so it's a bunch of rows, one output row for each step for each batch
    seq_output = tf.concat(outputs, axis=1)
    output = tf.reshape(seq_output, [-1, lstm_size])
    
    # Now connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(num_classes))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and batch
    logits = tf.matmul(output, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    preds = tf.nn.softmax(logits, name='predictions')
    
    # Reshape the targets to match the logits
    y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    cost = tf.reduce_mean(loss)

    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    # Export the nodes
    # NOTE: I'm using a namedtuple here because I think they are cool
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'preds', 'optimizer']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

## Training

In [17]:
samples_per_batch = 100
sample_length = 100 
lstm_size = 512
num_layers = 2
learning_rate = 0.002
keep_prob = 0.5

In [ ]:
epoch_count = 20
# Save every N iterations
save_every_n = 200

print('Generating training and validation data...')
train_x, train_y, val_x, val_y = split_data(chars, samples_per_batch, sample_length)
print('train_x len: {}'.format(len(train_x)))
print('train_y len: {}'.format(len(train_y)))
print('val_x len: {}'.format(len(val_x)))
print('val_y len: {}'.format(len(val_y)))
print('Data generation complete.')

print('Building model...')
model = build_rnn(len(vocab_int), 
                  samples_per_batch=samples_per_batch,
                  sample_length=sample_length,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)
print('Model built')

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    n_batches = len(train_x)
    print('total batches: {}'.format(n_batches))
    
    iteration_count = n_batches * epoch_count
    print('total iteration count: {}'.format(iteration_count))
    
    for epoch in range(epoch_count):
        
        new_state = sess.run(model.initial_state)
        loss = 0
        
        for i, (x, y) in enumerate(get_batch(train_x, train_y)):
#             print(train_x[i])
#             print(x.shape)
            iteration = epoch*n_batches + i + 1
            start = time.time()
            
            feed = {model.inputs: x, 
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            
            batch_loss, new_state, _ = sess.run([model.cost, model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{} '.format(epoch, epoch_count-1),
                  'Iteration {}/{}'.format(iteration, iteration_count),
                  'Training loss: {:.4f}'.format(loss/(i+1)),
                  '{:.4f} sec/batch'.format((end-start)))
            
            if (iteration%save_every_n == 0) or (iteration == iteration_count):
                # Check performance, notice dropout has been set to 1
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x, y in get_batch(val_x, val_y):
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    batch_loss, new_state = sess.run([model.cost, model.final_state], feed_dict=feed)
                    val_loss.append(batch_loss)

                print('Validation loss:', np.mean(val_loss),
                      'Saving checkpoint!')
                saver.save(sess, "checkpoints/i{}_l{}_v{:.3f}.ckpt".format(iteration, lstm_size, np.mean(val_loss)))
        

In [18]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i3560_l512_v1.330.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512_v2.251.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512_v1.872.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l512_v1.697.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l512_v1.579.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l512_v1.509.ckpt"
all_model_checkpoint_paths: "checkpoints/i1200_l512_v1.472.ckpt"
all_model_checkpoint_paths: "checkpoints/i1400_l512_v1.430.ckpt"
all_model_checkpoint_paths: "checkpoints/i1600_l512_v1.400.ckpt"
all_model_checkpoint_paths: "checkpoints/i1800_l512_v1.390.ckpt"
all_model_checkpoint_paths: "checkpoints/i2000_l512_v1.373.ckpt"
all_model_checkpoint_paths: "checkpoints/i2200_l512_v1.368.ckpt"
all_model_checkpoint_paths: "checkpoints/i2400_l512_v1.350.ckpt"
all_model_checkpoint_paths: "checkpoints/i2600_l512_v1.346.ckpt"
all_model_checkpoint_paths: "checkpoints/i2800_l512_v1.346.ckpt"
all_model_checkpoint_paths: "check

## Sampling

In [19]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [20]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = build_rnn(vocab_size, lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            #x[0,0] = ord(c)# vocab_to_int[c]
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab), 1)
        #samples.append(chr(c))#(int_to_vocab[c])
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            #samples.append(chr(c))# (int_to_vocab[c])
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [21]:
checkpoint = "checkpoints/i3560_l512_v1.330.ckpt"
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Fra")
print(samp)

InvalidArgumentError: Assign requires shapes of both tensors to match. lhs shape= [77] rhs shape= [83]
	 [[Node: save/Assign_19 = Assign[T=DT_FLOAT, _class=["loc:@softmax/Variable_1"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/cpu:0"](softmax/Variable_1/Adam_1, save/RestoreV2_19)]]

Caused by op 'save/Assign_19', defined at:
  File "/Users/ivan/anaconda3/envs/carnd-term1/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/ivan/anaconda3/envs/carnd-term1/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/ivan/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/ivan/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/ivan/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipyk